# How-To, First Steps

## Load package and create basic options

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from robustportfolio.finance import make_option

Let's start with some basic options. Remember that 1D PutOnMax is just Put (the same goes for CallOnMax)

In [ ]:
option1 = make_option(option_type='putonmax', strike=90)
option2 = make_option(option_type='putonmax', strike=80, payoff_dates = 5)
option3 = make_option(option_type='callonmax', strike=90, payoff_dates = [3,5])
option4 = make_option(option_type='put2call1') # American Put2Call1 option

Now let's see how these options got instantiated

In [ ]:
print('Option 1: {}'.format(option1))
print('Option 2: {}'.format(option2))
print('Option 3: {}'.format(option3))
print('Option 4: {}'.format(option4))

In [ ]:
print('Option 1 payoff at x = 80, 90, and 100, t = 3: {}'.format(option1.payoff([[80], [90], [100]], 3)))
print('Option 2 payoff at x = 100, t = 3: {}'.format(option2.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 3: {}'.format(option3.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 4: {}'.format(option3.payoff(100, 4)))

## Creating solver and solving some problems

Now let's create some basic 1D Problem with Rectangular multiplicative dynamics and no trading constraints. For that, we need module `robustportfolio.pricing`

In [ ]:
from robustportfolio.pricing import *
import numpy as np
pm1 = Problem(starting_price=np.array(100), 
            price_dynamics=ConstantDynamics(support=RectangularHandler([.9, 1.1]), type='mult'),
            trading_constraints=NoConstraints, option=option1, 
            lattice=Lattice(delta=[1]), 
            time_horizon=5)

Now we create solver with some parameters.

Most of the time, there is no point in tweaking *all* of these parameters, only some, namely `enable_timer` and `iter_tick`

In [ ]:
opts = {'convex_hull_filter': 'qhull', 'convex_hull_prune_fail_count': 0,
        'convex_hull_prune_success_count':0,'convex_hull_prune_corner_n': 3,'convex_hull_prune_seed': 0} 
solver = ConvhullSolver(enable_timer=True, pricer_options=opts, ignore_warnings=True, iter_tick=50)

Now we solve it and see the result $V_0(x_0)$

In [ ]:
sol1 = solver.solve(pm1)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Let's play around and change Lattice step

In [ ]:
pm1.lattice = Lattice(delta=[.1])
sol2 = solver.solve(pm1)
print('Value: {0}'.format(sol2['Vf'][0][0]))

Let's try 2D Problem with another option and *additive* dynamics.

In [ ]:
pm2 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 200
sol3 = solver.solve(pm2)
print('Value: {0}'.format(sol3['Vf'][0][0]))